# Preprocesar
En este archivo vamos a preprocesar el texto de manera que busquemos la mejor manera de sacar las etiquetas de cada tipo de palabra (POS tagging). El texto que vamos a usar va a estar en español, por lo que tenemos que usar modelos y métodos que lo soporten y tener en cuenta la sintaxis de dicho idioma. 

# 1 Sacar etiquetas
Como hemos mencionado antes, vamos a sacar las etiquetas de cada palabra en un texto, comprobaremos el funcionamiento de varios modelos y discutiremos con cuál nos quedamos.

## 1.1 Imports
El primer paso es definir los imports que vamos a usar durante esta parte, así como descargar datos para dichos modelos.

In [1]:
import spacy
import stanza
from flair.data import Sentence
from flair.models import SequenceTagger

La celda de abajo solo es necesaria una vez.

In [4]:
spacy.cli.download('es_core_news_sm')
print('-'*80)
spacy.cli.download('es_dep_news_trf')
print('-'*80)
stanza.download('es')

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
--------------------------------------------------------------------------------
✔ Download and installation successful
You can now load the package via spacy.load('es_dep_news_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
--------------------------------------------------------------------------------


2026-01-05 19:47:17 INFO: Downloaded file to C:\Users\Jorge\stanza_resources\resources.json
2026-01-05 19:47:17 INFO: Downloading default packages for language: es (Spanish) ...
2026-01-05 19:47:18 INFO: File exists: C:\Users\Jorge\stanza_resources\es\default.zip
2026-01-05 19:47:22 INFO: Finished downloading models and saved to C:\Users\Jorge\stanza_resources


## 1.2 Corpus de prueba
Antes de pasar a la acción definimos un corpus de prueba para evaluar los modelos.En este vamos a tener todas las frases que vayamos a usar a lo largo de todo el preprocesamiento


In [2]:
corpus = [
    'El gato come pescado.',
    'El equipo de fútbol jugó su partido. Este ganó con facilidad.',
    'El coche rojo se estropeó, así que lo llevé al taller.',
    'La presidenta y el director se reunieron; ella habló primero.',
    'Entregué el informe a la jefa después de que ella lo leyera.',
    'Los equipos trabajaron duro, y al final ellos ganaron el premio.',
    'A pesar de sus problemas, el artista terminó su obra.',
    'El hermano de María dijo que él vendría.',
    'En su oficina, el abogado revisó los documentos.',
    'El libro que leí es fascinante; este autor siempre sorprende.',
    'El gato persiguió al ratón, pero este logró escapar.',
    'Hablé con Pedro sobre su proyecto y luego él me envió los archivos.',
    'Ayer hablé con Juan, le dije: "dímelo, por favor", y no me lo quiso decir',
    'Dímelo',
    'Dí me lo',
    'Di me lo',
    'Él vino a dármelo',
    "La cabra de mi amigo Juan arremete todos los días contra la valla.",
    "Mi madre me dijo: 'cómete la tarta', por lo que me la comí.",
    "Mi profesor me dijo: 'comete este error y suspenderás', por lo que tuve mucho cuidado."
    "No sé si viste ayer a mi padre, pero yo sí.",
    "Vístete rápido, que ya nos vamos.",
    "Muéstrame tu chaqueta nueva."
]

## 1.3 Pruebas
Vamos a empezar a probar modelos sobre todas las frases que hemos definido antes.

### 1.3.1 Spacy - es_core_news_sm

In [10]:
nlp = spacy.load("es_core_news_sm")
test1 = []
for text in corpus: test1.append(nlp(text))
for doc in test1:
    for token in doc:
        print(token.text, token.pos_)
    print()

El DET
gato NOUN
come VERB
pescado ADJ
. PUNCT

El DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
Este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

El DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
al ADP
taller NOUN
. PUNCT

La DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

Entregué PROPN
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

Los DET
equipos NOUN
trabajaron VERB
duro ADJ
, PUNCT
y CCONJ
al ADP
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

A ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

El DET
hermano NOUN
de ADP
María PROPN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

En ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

El DET
libro NOU

En general lo hace muy bien, aunque confunde VERB que comienzan una oración con PROPN (sustantivos propios) y, en la primera oración, confunde el sustantivo pescado por el adjetivo, por lo que podemos intuir que cometerá más veces ese error. Por lo que a nuestra futura tarea respecta, el primer error puede ser garrafal, pues podría pensar el modelo que un pronombre se refiere a un verbo de esos mal etiquetados. Además, el verbo decir más los enclíticos me y lo lo considera un sustantivo, fallo enorme ya que son dos pronombres tras el verbo.
Veamos el resto de modelos.

### 1.3.2 Spacy - es_dep_news_trf

In [11]:
nlp = spacy.load("es_dep_news_trf")
test2 = []
for text in corpus: test2.append(nlp(text))
for doc in test2:
    for token in doc:
        print(token.text, token.pos_)
    print()

El DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

El DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
Este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

El DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
al ADP
taller NOUN
. PUNCT

La DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

Entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

Los DET
equipos NOUN
trabajaron VERB
duro ADV
, PUNCT
y CCONJ
al ADP
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

A ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

El DET
hermano NOUN
de ADP
María PROPN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

En ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

El DET
libro NOU

En este caso vemos que corrige todos los errores previos, salvo el caso de los enclíticos, que los considera verbos (bien, pero solo la raíz es el verbo). Esto muestra que este modelo es mejor, pero no perfecto, pues confunde, por ejemplo "Dí", de separar los enclíticos en "Dímelo", por AUX, pero sin la tilde dice correctamente que es un verbo, esto lo tendremos en cuenta más adelante.

### 1.3.3 Spacy - es_dep_news_trf (texto en minúsculas)

In [12]:
nlp = spacy.load("es_dep_news_trf")
test3 = []
for text in corpus: test3.append(nlp(text.lower()))
for doc in test3:
    for token in doc:
        print(token.text, token.pos_)
    print()

el DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

el DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

el DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
al ADP
taller NOUN
. PUNCT

la DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

los DET
equipos NOUN
trabajaron VERB
duro ADV
, PUNCT
y CCONJ
al ADP
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

a ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

el DET
hermano NOUN
de ADP
maría PROPN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

en ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

el DET
libro NOU

Al hacer varias pruebas nos damos cuenta que estos modelos son <em>case sensitive</em>, es decir, las palabras en mayúsculas y minúsculas importan. En este caso vemos que se solventa el problema del AUX en "dí", pero aún así palabras como "dímelo" no las hace bien. Probemos con otro modelo, en este caso Stanza.

### 1.3.4 Stanza

In [ ]:
nlp = stanza.Pipeline('es', verbose=False)
test4 = []
for text in corpus: test4.append(nlp(text))
for doc in test4:
    for sentence in doc.sentences:
        for word in sentence.words:
            print(word.text, word.pos)
    print()

El DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

El DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
Este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

El DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
a ADP
el DET
taller NOUN
. PUNCT

La DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

Entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

Los DET
equipos NOUN
trabajaron VERB
duro ADJ
, PUNCT
y CCONJ
a ADP
el DET
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

A ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

El DET
hermano NOUN
de ADP
María PROPN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

En ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

El D

En este caso el modelo funciona muy similar a Spacy con es_dep_news_trf. Tampoco identifica correctamente los pronombres en "dímelo", y en este caso lo marca como interjección (INTJ). Cuando solo ponemos el verbo con enclíticos (Dímelo) lo considera verbo, y cuando lo separamos con espacios las partes analiza bien los pronombres pero el verbo vuelve a confundirlo con interjección, aunque si el verbo está sin tilde lo sigue marcanto como INTJ. Probemos con el texto en minúsculas. 

### 1.3.5 Stanza (minúsculas)

In [50]:
nlp = stanza.Pipeline('es', verbose=False)
test5 = []
for text in corpus: test5.append(nlp(text.lower()))
for doc in test5:
    for sentence in doc.sentences:
        for word in sentence.words:
            print(word.text, word.pos)
    print()

el DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

el DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

el DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
a ADP
el DET
taller NOUN
. PUNCT

la DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

los DET
equipos NOUN
trabajaron VERB
duro ADJ
, PUNCT
y CCONJ
a ADP
el DET
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

a ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

el DET
hermano NOUN
de ADP
maría NOUN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

en ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

el DE

Aquí cambia ligeramente: "di" lo marca bien como verbo, mientras que "dímelo" lo marca como sustantivo, y lo demás se mantiene igual. Aunque sea parecido a Spacy, este parece menos consistente con verbos con enclíticos.

### 1.3.6 Stanza (uso de processors)

In [8]:
nlp = stanza.Pipeline(
    "es",
    processors="tokenize,mwt,pos,lemma",
    tokenize_no_ssplit=False, 
    verbose=False
)

for text in corpus:
    doc = nlp(text)
    for sent in doc.sentences:
        for w in sent.words:
            print(w.text, w.lemma, w.upos)
        print()

El el DET
gato gato NOUN
come comir VERB
pescado pescado NOUN
. . PUNCT

El el DET
equipo equipo NOUN
de de ADP
fútbol fútbol NOUN
jugó jugar VERB
su su DET
partido partido NOUN
. . PUNCT

Este este PRON
ganó ganar VERB
con con ADP
facilidad facilidad NOUN
. . PUNCT

El el DET
coche coche NOUN
rojo rojo ADJ
se él PRON
estropeó estropear VERB
, , PUNCT
así así ADV
que que SCONJ
lo él PRON
llevé llever VERB
a a ADP
el el DET
taller taller NOUN
. . PUNCT

La el DET
presidenta presidenta NOUN
y y CCONJ
el el DET
director director NOUN
se él PRON
reunieron reunir VERB
; ; PUNCT
ella él PRON
habló hablar VERB
primero primero ADV
. . PUNCT

Entregué entreguar VERB
el el DET
informe informe NOUN
a a ADP
la el DET
jefa jefa NOUN
después después ADV
de de ADP
que que SCONJ
ella él PRON
lo él PRON
leyera leerar VERB
. . PUNCT

Los el DET
equipos equipo NOUN
trabajaron trabajar VERB
duro duro ADJ
, , PUNCT
y y CCONJ
a a ADP
el el DET
final final NOUN
ellos él PRON
ganaron ganar VERB
el el DET
prem

Vemos que este modelo no es mucho mejor que lo anterior, no es capaz de separar los enclíticos de los verbos. Vamos ahora con la última prueba, Flair.

### 1.3.7 Flair

In [20]:
# Cargar modelo POS (multilingual)
tagger = SequenceTagger.load("pos-multi")

for text in corpus:
    sentence = Sentence(text.lower())
    tagger.predict(sentence)
    for token in sentence:
        print(token.text, token.get_labels()[0].value)
    print()

2026-01-05 20:03:17,324 SequenceTagger predicts: Dictionary with 17 tags: NOUN, PUNCT, ADP, VERB, ADJ, DET, PROPN, ADV, PRON, AUX, CCONJ, NUM, SCONJ, PART, X, SYM, INTJ
el DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

el DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

el DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que CCONJ
lo PRON
llevé VERB
al DET
taller NOUN
. PUNCT

la DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADJ
. PUNCT

entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

los DET
equipos NOUN
trabajaron VERB
duro ADJ
, PUNCT
y CCONJ
al ADP
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

a ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

el DET
hermano NOUN
de ADP
maría NOU

En este modelo vemos más inconsistencias todavía. "di" lo marca como ADP, y "lo" al separar "dímelo" lo marca como verbo. No mejora en ningún aspecto a los anteriores. <br>
Para mejorar este comportamiento vamos a hacer un preprocesamiento personalizado, el cual tratará de separar los enclíticos, pero para ello tenemos que ver qué verbos tienen enclíticos y cuáles no.

# 2 Preprocesamiento personalizado
Tras varias pruebas pensamos que Stanza es el modelo más consistente, ya que solo falla en los enclíticos, aunque tiene un multi-word-token (mwt). Para mejorar con los enclíticos podemos hacer nuestro propio preprocesamiento al texto para que así lo etiquete como debe.  
En primer lugar vamos a probar a utilizar un stemmer para sacar la raíz de las palabras, de esta forma vamos a ver si las etiquetas se mantienen como antes y, además, resuelve los verbos con enclíticos y los verbos con falsos enclíticos (i.e. verbos como "comete", "viste", etc.). Idealmente queremos que se lematicen los verbos sin enclíticos y los que sí los tienen los deje igual para luego separarlos. 
Si esto no funciona como esperamos entonces cogeremos la lista de pronombres que pueden ir junto a los verbos y los ponemos en orden (i.e. nunca se tiene "lasme", siempre "melas", como en "dámelas"). Pero esto solo no es suficiente, puesto que hay verbos que tienen "me", "te" u otros. Pero comete es diferente de cómete, de comer, por lo que necesitamos una lista con reglas por si nos encontramos ciertos verbos. 

## 2.1 Imports
Primero definimos los imports que usaremos en esta parte, al igual que hicimos antes.

In [9]:
from nltk.stem import SnowballStemmer, LancasterStemmer
from nltk.tokenize import word_tokenize
import re
import json

## 2.2 Funciones auxiliares
Para no repetir código, vamos a definir una o varias funciones auxiliares.

In [78]:
def compare_tags(docs: list, compare: list):
    for doc, cmp in zip(docs, compare):
        for doc_sentence, cmp_sentence,  in zip(doc.sentences, cmp.sentences):
            for doc_word, cmp_word in zip(doc_sentence.words, cmp_sentence.words):
                if doc_word.pos != cmp_word.pos:
                    print('-'*40) 
                    print(doc_word.text, doc_word.pos, '|', cmp_word.text, cmp_word.pos)
                    print('-'*40)
                else:
                    print(doc_word.text, doc_word.pos, '|', cmp_word.text, cmp_word.pos)
            print() 

In [67]:
def print_tags(docs: list = [], compare: list | None = None):
    if compare:
        compare_tags(docs, compare)
    else:
        for doc in docs:
            for sentence in doc.sentences:
                for word in sentence.words:
                    print(word.text, word.pos)
                print()    

## 2.3 Pruebas
Vamos ahora a probar los modelos para arreglar el fallo con los enclíticos.

### 2.3.1 LancasterStemmer

In [79]:
tokens_list = [word_tokenize(text.lower(), language="spanish") for text in corpus]
stemmer = LancasterStemmer()
stems = []
for token_phrase in tokens_list:
    stems.append([stemmer.stem(token) for token in token_phrase])
stems=[' '.join(stem) for stem in stems]
    
nlp = stanza.Pipeline(
    lang="es",
    processors="tokenize,mwt,pos,lemma,depparse",
    tokenize_pretokenized=False,
    use_gpu=True,
    verbose=False
)

docs = [nlp(stem) for stem in stems]
print_tags(docs, test5)

el DET | el DET
gato NOUN | gato NOUN
----------------------------------------
com PROPN | come VERB
----------------------------------------
pescado NOUN | pescado NOUN
. PUNCT | . PUNCT

el DET | el DET
equipo NOUN | equipo NOUN
de ADP | de ADP
fútbol NOUN | fútbol NOUN
jugó VERB | jugó VERB
su DET | su DET
partido NOUN | partido NOUN
. PUNCT | . PUNCT

----------------------------------------
est PROPN | este PRON
----------------------------------------
ganó VERB | ganó VERB
con ADP | con ADP
facilidad NOUN | facilidad NOUN
. PUNCT | . PUNCT

el DET | el DET
coch NOUN | coche NOUN
rojo ADJ | rojo ADJ
se PRON | se PRON
estropeó VERB | estropeó VERB
, PUNCT | , PUNCT
así ADV | así ADV
que SCONJ | que SCONJ
lo PRON | lo PRON
llevé VERB | llevé VERB
a ADP | a ADP
el DET | el DET
----------------------------------------
tal PRON | taller NOUN
----------------------------------------
. PUNCT | . PUNCT

la DET | la DET
president NOUN | presidenta NOUN
y CCONJ | y CCONJ
el DET | el DET
dir

Vemos que en general ha empeorado notablemente el resultado, ya que al pasar loss stems, las palabras pierden parte del significado. Vamos a hacer una última prueba con stemmers antes de descartarlos.

### 2.3.2 SnowballStemmer

In [80]:
tokens_list = [word_tokenize(text.lower(), language="spanish") for text in corpus]
stemmer = SnowballStemmer("spanish")
stems = []
for token_phrase in tokens_list:
    stems.append([stemmer.stem(token) for token in token_phrase])
stems=[' '.join(stem) for stem in stems]
    
nlp = stanza.Pipeline(
    lang="es",
    processors="tokenize,mwt,pos,lemma,depparse",
    tokenize_pretokenized=False,
    use_gpu=True,
    verbose=False
)
docs = [nlp(stem) for stem in stems]
print_tags(docs, test5)

el DET | el DET
gat NOUN | gato NOUN
----------------------------------------
com PROPN | come VERB
----------------------------------------
pesc NOUN | pescado NOUN
. PUNCT | . PUNCT

el DET | el DET
equip NOUN | equipo NOUN
de ADP | de ADP
futbol NOUN | fútbol NOUN
----------------------------------------
jug PROPN | jugó VERB
----------------------------------------
su DET | su DET
part NOUN | partido NOUN
. PUNCT | . PUNCT

el DET | el DET
----------------------------------------
coch PROPN | coche NOUN
----------------------------------------
----------------------------------------
roj PROPN | rojo ADJ
----------------------------------------
se PRON | se PRON
estrope VERB | estropeó VERB
, PUNCT | , PUNCT
asi ADV | así ADV
que SCONJ | que SCONJ
lo PRON | lo PRON
llev VERB | llevé VERB
a ADP | a ADP
el DET | el DET
tall NOUN | taller NOUN
. PUNCT | . PUNCT

la DET | la DET
president NOUN | presidenta NOUN
y CCONJ | y CCONJ
el DET | el DET
director NOUN | director NOUN
se PRON | s

Este caso funciona igual que el anterior. Por lo que descartamos preprocesar el texto con stemmers.

## 2.4 Lista de excepciones + reglas
Como podemos observar, stanza tiene bastantes carencias a la hora de reconocer y separar los clíticos en español siendo que solo es bueno reconociendo los infinitivos con clíticos y los gerundios cuando solo tienen un clítico, si presentan doble clítico no los reconoce. Por eso, cualquier forma verbal en imperativo con clíticos fallará a la hora de reconocerla. Tampoco nos ha funcionado el stemmer para nuestro propósito, por lo que pasamos a hacer un procesado puramente manual, llevar una serie de reglas para separar verbos.

#### EJEMPLO VERBO JUGAR

| Forma verbal | Descripción | Stanza (salida incorrecta) | Deseado (análisis correcto) |
|-------------|-------------|----------------------------|-----------------------------|
| juégalo | Imperativo 2ª persona singular + 1 clítico | juégalo → juégalo (NOUN) | juega (jugar, VERB) + lo (él, PRON) |
| juégatelo | Imperativo 2ª persona singular + 2 clíticos | juégatelo → juégatelo (NOUN) | juega (jugar, VERB) + te (tú, PRON) + lo (él, PRON) |
| jugadle | Imperativo 2ª persona plural + 1 clítico | jugadle → jugadle (NOUN) | jugad (jugar, VERB) + le (él, PRON) |
| jugádnoslo | Imperativo 2ª persona plural + 2 clíticos | jugadno → jugadno (NOUN) + lo (él, PRON) | jugad (jugar, VERB) + nos (nosotros, PRON) + lo (él, PRON) |
| juéguelo | Imperativo 2ª persona singular (usted) + 1 clítico | juéguelo → juéguelo (NOUN) | juegue (jugar, VERB) + lo (él, PRON) |
| juégueselo | Imperativo 2ª persona singular (usted) + 2 clíticos | juégueselo → juégueselo (NOUN) | juegue (jugar, VERB) + se (él, PRON) + lo (él, PRON) |
| jueguenlo | Imperativo 2ª persona plural (ustedes) + 1 clítico | jueguenlo → jueguenlo (NOUN) | jueguen (jugar, VERB) + lo (él, PRON) |
| jueguenselo | Imperativo 2ª persona plural (ustedes) + 2 clíticos | jueguenselo → jueguenselo (NOUN) | jueguen (jugar, VERB) + se (él, PRON) + lo (él, PRON) |
| juguémoslo | Imperativo 1ª persona plural + 1 clítico | juguémoslo → juguémoslo (NOUN) | juguemos (jugar, VERB) + lo (él, PRON) |
|juguémonoslo | Imperativo 1ª persona plural + 2 clítico | juguémonoslo -> juguémonoslo NOUN |juguemos (jugar, VERB) + nos (nosotros, PRON) + lo (él, PRON)|
jugándomelo | Gerundio + 2 clíticos | jugando -> jugar VERB + melo -> melo NOUN | jugando (jugar, VERB) + me (yo, PRON) + lo (él, PRON)|

Para ello meteremos estas formas verbales en una lista de excepciones para separarlas de forma manual y así pueden ser tokenizadas correctamente

#### LISTA DE VERBOS CON CLÍTICOS

Lista de excepciones que el tokenizador de stanza no sabe tokenizar, típicamente son las formas imperativas de los verbos en español que presentan clíticos.

Los verbos en la lista se reconocen mediante expresiones regulares de la base de la palabra y todas las combinaciones de clíticos que nos podemos encontrar en ella. También se especifica de que verbo viene ya que la base puede coincidir con alguna forma verbal de algún otro verbo como sucede con "díselo" cuya base es di que coincide con el pasado del verbo "dar". Por eso se le indica el lema al que pertenece y el upos ya que aunque todos sean verbos, el tokenizador no lo reconoce e intenta adivinarlo dándole alguna upos aleatoria.

In [10]:
with open('patterns.json', 'r', encoding='utf-8') as f:
    patterns = json.load(f)

Lista de clíticos del español

In [11]:
CLITICS = ["me", "te", "se", "nos", "os", "lo", "la", "le", "los", "las", "les"]

**split_clitics**

Separa los clíticos de la palabra original

In [12]:
def split_clitics(clitic_string: str):
    clitics = []
    remaining = clitic_string

    for cl in CLITICS:
        if remaining.startswith(cl):
            clitics.append(cl)
            remaining = remaining[len(cl):]

    return clitics

**preprocess_imperatives**

Coordina el proceso de limpieza del texto. Se descompone en 3 fases:

 1º- Inicialización donde crea una lista (imperative_meta) que guardará la información de cada verbo que se modifique

 2º- El Ciclo de Transformación donde recorrerá el IMPERATIVE_MAP con los patrones y las reglas de sustitución

 3º- Actualización donde aplicará los cambios al texto original devolviéndolo junto con sus metadatos recolectados

**make_replacer**

Se corresponde con el ciclo de transformación y se encargará de coger cada patrón junto con su regla asociada del IMPERATIVE_MAP. Además también devuelve la expresión regular lista para emplearse y la función replacer configurada.

**replacer**

Función que no llama el usuario sino que la llama el motor de expresiones regulares de python cada vez que encuentra una coincidencia que encaja con el patrón. Su funcionamiento es el siguiente:

 1º- Captura los clíticos de la palabra, si no los hay, devuelve la palabra sin modificar

 2º- Crea una nueva cadena de texto separando la raíz del verbo de los pronombre empleando espacios

 3º- Guarda en imperative_meta la información técnica del verbo para que no se pierda la información que había aunque se cambie la palabra

 4º- Asegura que respeta las mayúsculas que la palabra original tenía

 5º- Devuelve la cadena separada para que se inserte en el texto originial

In [13]:
def preprocess_imperatives(text: str):
    imperative_meta = []

    def make_replacer(pattern: str, rule: dict):
        regex = re.compile(pattern, flags=re.IGNORECASE) # Carga la expresión regular del patrón

        def replacer(match): # Si no coincide con el patrón, devuelve la palabra sin modificar
            groups = match.groups() # Analiza el patrón para buscar que clíticos tiene
            # Mete los clíticos encontrados como elementos de una lista
            clitic_string = "".join(g for g in groups if g) if groups else ""
            clitics = split_clitics(clitic_string)

            # Si no hay clíticos, no tocar la palabra original
            if not clitics:
                return match.group(0)
            
            # Aplicar la regla correspondiente si hay clíticos
            replacement = " ".join([rule["base"]] + clitics)

            imperative_meta.append({
                "base": rule["base"],
                "lemma": rule["lemma"],
                "upos": rule["upos"]
            })

            # Mantener mayúscula inicial
            if match.group(0)[0].isupper():
                replacement = replacement.capitalize()

            return replacement

        return regex, replacer

    for pattern, rule in patterns.items():
        regex, replacer = make_replacer(pattern, rule)
        text = regex.sub(replacer, text) # Aplica la sustitución al texto original

    return text, imperative_meta

**process_text**

Función que separá los clíticos llamando a preprocess_imperatives para despues pasarlos por el tokenizador de stanza.pipeline y así tener ya cada palabra tokenizada para luego pasarla al formato final

In [14]:
def process_text(text, nlp):
    """Procesa un solo texto."""
    preprocessed, imperative_m = preprocess_imperatives(text)
    doc = nlp(preprocessed.lower())
    return doc, imperative_m

In [15]:
def parse_corpus(corpus):
    # Crear modelo
    nlp = stanza.Pipeline(
        lang="es",
        processors="tokenize,mwt,pos,lemma,depparse",
        tokenize_pretokenized=False,
        use_gpu=True,
        verbose=False
    )
    
    docs = []
    imperative_metas = []
    
    for text in corpus:
        doc, imperative_meta = process_text(text, nlp)
        docs.append(doc)
        imperative_metas.append(imperative_meta)    
    
    return docs, imperative_metas

In [16]:
doc, imperative_meta = parse_corpus(corpus)

In [18]:
for d in doc:
    for sentence in d.sentences:
        for token in sentence.words:
            print(token.text, token.pos)
        print() 

el DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

el DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT

este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

el DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
a ADP
el DET
taller NOUN
. PUNCT

la DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

los DET
equipos NOUN
trabajaron VERB
duro ADJ
, PUNCT
y CCONJ
a ADP
el DET
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

a ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

el DET
hermano NOUN
de ADP
maría NOUN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

en ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

el D

Vemos que este método ha sido todo un éxito, pero no ha sido moco de pavo crear tantas reglas para el preprocesamiento. Además, es seguro que nos dejamos muchos verbos, ya que es imposible tener en cuenta todos, pero sí pensamos que este preprocesado es bastante efectivo.

# 3 Dataset
Ahora vamos a buscar un dataset para sacar sus etiquetas part-of-speech (POS TAGs). Este dataset será el que después probaremos qué tal sirve nuestro modelo para resolver las referencias de los pronombres a los sustantivos. Hemos encontrado en Hugging Face un dataset llamado CulturaX, que tiene datos en más de 160 idiomas, entre ellos el español. Consideramos que es un dataset lo suficientemente completo con multitud de textos de diferentes procedencias para usarlo en nuestro proyecto. Vamos a coger un subset de varios miles de textos ya que el completo tiene millones o más, y no necesitamos tantos.

## 3.1 Imports
Vamos a ver los imports que necesitamos y el login de Hugging Face.
Si se quiere ejecutar la celda de abajo hay que seguir las instrucciones del login en notebook, es bastante intuitivo.

In [1]:
from huggingface_hub import notebook_login
notebook_login()

from datasets import load_dataset
import json

## 3.2 Cargar textos
Ahora vamos a cargar los textos en una variable 'ds_es', la cual funciona como Stream para no descargar los datos, solo metadatos que permiten su uso, y después vamos a tomar trivialmente 5 mil textos para nuestro proyecto.

In [14]:
ds_es = load_dataset(
    "uonlp/CulturaX",  # Nombre del dataset
    "es",              # Config de idioma español
    streaming=True     # Stream para no descargar completo
)

Resolving data files:   0%|          | 0/512 [00:00<?, ?it/s]

In [15]:
# Tomar solo los primeros n ejemplos
n = 5000  
subset = ds_es["train"].take(n)

# Convertir a lista si quieres verlos o trabajar con ellos
data = list(subset)

print(len(data))  # debería dar n

5000


## 3.3 Mostrar datos
Para ver un poco los datos vamos a mostrar unos cuantos textos, primero veamos que tipo es cada cosa y cómo podemos acceder a dichos textos.

In [17]:
print(type(data[1]))

<class 'dict'>


In [18]:
print(data[1].keys())

dict_keys(['text', 'timestamp', 'url', 'source'])


Vemos que tiene 4 claves. Veamos cada campo, excepto texto.

In [21]:
print(data[1]['url'])
print(data[1]['source'])
print(data[1]['timestamp'])

https://periodicoalminutosv.com/5072/nacionales/presidente-bukele-es-bien-evaluado-en-los-primeros-meses-de-su-gestion-90-de-la-poblacion-opina-que-esta-ayudando/
OSCAR-2109
2021-03-04T07:09:46Z


Se entiende bastante bien qué es cada cosa, excepto la fuente. Vamos a mostrar ahora los primeros 5 textos.

In [28]:
for i in range(5):
    print(data[i]['text'], '\n')
    

Gobierno confía en que Corte Constitucional dé vía libre al mecanismo fast track - Eje21
Gobierno confía en que Corte Constitucional dé vía libre al mecanismo fast track
Bogotá, 04 de diciembre _ RAM_ Así lo confirmó este sábado el ministro del Interior Juan Fernando Cristo, quien señaló que: "el Gobierno emprenderá el camino de implementación del nuevo acuerdo de paz que decida la Corte Constitucional", y confía en que la decisión sobre la vía rápida para la implementación de las leyes de paz se tome antes de la vacancia judicial y el fin de esta legislatura en el Congreso, "tenemos la fe de que va a dar salidas que garanticen implementación rápida de los acuerdos". 

hola yo estoy pasando por la misma situacion el 19 de diciembre pedi en mandarake ub shin mazinger , armadura blindada para vf 25 1/60 y el set para el vf 25 tornado y hasta el dia de hoy la pgina de correos de mexico solo dice recibido en oficina postal 2 evntos, la verdad no se para cuando llegue mi paquete es la prime

Podemos ver que hay algunos textos con erratas, y con lenguas de hispanoamérica, lo cual puede ser interesante para ver el desempeño del etiquetado.

## 3.3 Almacenar datos
Ahora vamos a guardar los datos en un archivo y en el siguiente notebook sacaremos las etiquetas.

In [ ]:
filename = "CulturaX.json"
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)
print(f"Archivo guardado correctamente como {filename}")

# 4 Conclusiones
Para concluir este notebook vamos a comentar qué modelos y procesos nos han funcionado mejor. Para empezar, entre los modelos de POS TAG el mejor pensamos que es Stanza, que funciona muy parecido a SpaCy, pero Stanza cuenta con mwt como hemos comentado antes, además del resto de processors, por lo que lo consideramos ligeramente mejor. Además, hemos probado a atacar el texto de varias maneras, con tokenizadores y stemmers, lo cual no funcionó, y con nuestras propias reglas, lo cual ha resultado ser bastante bueno. Concluimos con que para aplicar el preprocesado a nuestro dataset 'CulturaX' primero le aplicaremos las reglas, después trataremos de separar los enclíticos a mano, y por último sacaremos las etiquetas con Stanza.